In [2]:
import pandas as pd

In [3]:
import torch
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from torch.utils.data import DataLoader

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [4]:
# exp004
import dataclasses
import torch
import pandas as pd
from sklearn.model_selection import KFold
from transformers import AutoModel, AutoTokenizer, AutoModelForMultipleChoice
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
from datasets import Dataset
from typing import Optional, Union

import transformers
import wandb
from datetime import datetime as dt
import os
import numpy as np
import tqdm

import logging 
from logging import Logger



@dataclasses.dataclass
class BertConfig:
    
    experiment_name: str
    dataset_dir: str
    
    debug: bool = False

    lr: float = 1e-5
    model_name: str = "OpenAssistant/reward-model-deberta-v3-large-v2"
    num_context: int = 3
    max_length: int = 512
    batch_size: int = 2
    epochs: int = 10
    iters_to_accumlate: int = 8
    weight_decay: float = 0.01
    warmup_ratio: float = 0.1
    
    freeze_embeddings: bool = True
    freeze_layers: int = 18
    reinitialize_layers: int = 0
    
    assume_completely_retrieved: bool = False
    n_samples: int = None
    steps: int = 100
    
    lora_r: float = 2
    lora_alpha: float = 4
    lora_dropout: float = 0.1
    use_peft: bool = False

def get_logger(
    output_dir: str,
):
    """
    logger を作成する. formatter は "%Y-%m-%d %H:%M:%S" で作成する.
    """
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.DEBUG)
    
    # formatter
    formatter = logging.Formatter("%(asctime)s %(levelname)s %(message)s")
    
    # handler
    handler = logging.StreamHandler()
    handler.setLevel(logging.DEBUG)
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    
    handler = logging.FileHandler(f"{output_dir}/log.txt", "w")
    handler.setLevel(logging.DEBUG)
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    
    return logger


@dataclasses.dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
        label_name = 'label' if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch


        
def preprocess_df(df, config):
    df["context"] = ""
    for i in range(config.num_context):
        df["context"] += df[f"searched_wiki_id_{i}"].astype(str) + "\n\n"
    
    for col in ["A", "B", "C", "D", "E"]:
        df[col] = df[col].fillna("")
    return df[["prompt", "context", "A", "B", "C", "D", "E", "answer"]]


def map_at_3(predictions, labels):
    map_sum = 0
    pred = np.argsort(-1*np.array(predictions),axis=1)[:,:3]
    for x,y in zip(pred,labels):
        z = [1/i if y==j else 0 for i,j in zip([1,2,3],x)]
        map_sum += np.sum(z)
    return map_sum / len(predictions)

def compute_metrics(p):
    predictions = p.predictions.tolist()
    labels = p.label_ids.tolist()
    return {"map@3": map_at_3(predictions, labels)}


In [5]:
df_test = pd.concat([
    pd.read_feather("../output/context_pipeline/stage1/exp009.py/20231005102738_gte-small_wikiall_without_sep_targetprompt_and_choice_without_sep_token_length90_stride_sentence3_drop_categoryTrue_all/train.feather"),
    pd.read_feather("../output/context_pipeline/stage1/exp009.py/20231005102738_gte-small_wikiall_without_sep_targetprompt_and_choice_without_sep_token_length90_stride_sentence3_drop_categoryTrue_all/test.feather"),
])

In [6]:
df_test["data_id"] = np.arange(len(df_test))

In [7]:
df_test["gpt4"] = df_test["gpt4"].fillna(df_test["answer"])
df_test["answer"] = df_test["gpt4"]

In [8]:
df_test.columns

Index(['prompt', 'A', 'B', 'C', 'D', 'E', 'answer', 'gpt4', 'wiki_title',
       'wiki_link', 'dataset', 'searched_ids', 'searched_text_ids',
       'searched_text', 'similarity', 'similarity_rank', 'searched_wiki_id_0',
       'searched_wiki_id_1', 'searched_wiki_id_2', 'searched_wiki_id_3',
       'searched_wiki_id_4', 'searched_wiki_id_5', 'searched_wiki_id_6',
       'searched_wiki_id_7', 'searched_wiki_id_8', 'searched_wiki_id_9',
       'searched_wiki_id_10', 'searched_wiki_id_11', 'searched_wiki_id_12',
       'searched_wiki_id_13', 'searched_wiki_id_14', 'searched_wiki_id_15',
       'searched_wiki_id_16', 'searched_wiki_id_17', 'searched_wiki_id_18',
       'searched_wiki_id_19', 'searched_wiki_id_20', 'searched_wiki_id_21',
       'searched_wiki_id_22', 'searched_wiki_id_23', 'searched_wiki_id_24',
       'searched_wiki_id_25', 'searched_wiki_id_26', 'searched_wiki_id_27',
       'searched_wiki_id_28', 'searched_wiki_id_29', 'searched_wiki_id_30',
       'searched_wiki_id_31'

In [9]:
df_tests = []

In [10]:
def make_df(indices, tta_id):
    df_ret = []
    for j, index in enumerate(indices):
        df = df_test.copy()
        df["tta_id"] = tta_id
        df["tta_no"] = j
        for i, idx in enumerate(index):
            df[f"searched_wiki_id_{i}"] = df[f"searched_wiki_id_{idx}"]
        df_ret.append(df)
    return pd.concat(df_ret)

In [11]:
indices = np.concatenate([[[0]] * 6, np.arange(1, 31).reshape(6, 5)], axis=1)
print(indices)
df_tests.append(make_df(indices=indices, tta_id="first_order"))

[[ 0  1  2  3  4  5]
 [ 0  6  7  8  9 10]
 [ 0 11 12 13 14 15]
 [ 0 16 17 18 19 20]
 [ 0 21 22 23 24 25]
 [ 0 26 27 28 29 30]]


In [12]:
df_tests = pd.concat(df_tests)

In [13]:
logger = get_logger(output_dir="")
logger.info("load data")

2023-10-05 11:40:40,595 INFO load data


In [14]:
def calc_map3(
    df_test,
    model_name,
    max_length,
    num_content,
):
    option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
    label = df_test["answer"].map(option_to_index).values
    logger = get_logger(output_dir="")
    logger.info("load data")
    
    config = BertConfig(
        debug=False,
        batch_size=1,
        experiment_name=f"",
        dataset_dir="",
        model_name=model_name,
        max_length=max_length,
        num_context=num_content,
    )
    df_test = preprocess_df(df_test, config)
    df_test = df_test[df_test["answer"].isin(["A", "B", "C", "D", "E"])]
    
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
    index_to_option = {v: k for k,v in option_to_index.items()}
    def preprocess(example):
        first_sentence = [ "[CLS] " + example['context'] ] * 5
        second_sentences = [" #### " + example['prompt'] + " [SEP] " + example[option] + " [SEP]" for option in 'ABCDE']
        tokenized_example = tokenizer(first_sentence, second_sentences, truncation="only_first", 
                                      max_length=config.max_length, add_special_tokens=False)
        tokenized_example['label'] = option_to_index[example['answer']]
        return tokenized_example

    test_dataset = Dataset.from_pandas(df_test)
    tokenized_test_dataset = test_dataset.map(preprocess, remove_columns=["prompt", "context", "A", "B", "C", "D", "E", "answer", "__index_level_0__"])

    data_collator = DataCollatorForMultipleChoice(tokenizer=tokenizer)
    test_dataloader = DataLoader(tokenized_test_dataset, batch_size=1, shuffle=False, collate_fn=data_collator)
    
    model = AutoModelForMultipleChoice.from_pretrained(config.model_name).cuda()
    
    test_predictions = []
    for batch in tqdm.tqdm(test_dataloader):
        with torch.autocast("cuda"):
            for k in batch.keys():
                batch[k] = batch[k].cuda()
            with torch.no_grad():
                outputs = model(**batch)
            test_predictions.append(outputs.logits.cpu().detach())

    test_predictions = torch.cat(test_predictions)
    test_predictions = test_predictions.numpy()
    
    print(map_at_3(test_predictions, label))
    return {
        "model_name": model_name,
        "max_length": max_length,
        "num_content": num_content,
        "map3": map_at_3(test_predictions, label),
        "predictions": test_predictions
    }

In [15]:
ret = []

In [16]:
ret.append(calc_map3(
    df_test=df_tests,
    model_name="../output/stage2/exp005.py/20231004123719_gtesmall_l90s3_maxlen256_bs2_lr1e-06_3epochs/fold0/",
    max_length=640,
    num_content=6,
))

2023-10-05 11:40:40,624 INFO load data
2023-10-05 11:40:40,624 INFO load data


  0%|          | 0/3000 [00:00<?, ?ex/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3000/3000 [08:51<00:00,  5.64it/s]


0.8667222222222222


In [25]:
df_tests2 = df_tests.copy()

In [18]:
for i, col in enumerate("ABCDE"):
    df_tests2[f"pred_{col}"] = ret[0]["predictions"][:, i]

In [19]:
df_tests2

,prompt,A,B,C,D,E,answer,gpt4,wiki_title,wiki_link,...,id,data_id,tta_id,tta_no,context,pred_A,pred_B,pred_C,pred_D,pred_E
0,What is the method of transcription in the lif...,RNA-templated transcription is the method of t...,Transcription occurs through a unique mechanis...,Reverse transcription is the method of transcr...,DNA-templated transcription is the method of t...,Transcription does not occur in the life cycle...,D,D,Becurtovirus,https://en.wikipedia.org/wiki/Becurtovirus,...,NaN,0,first_order,0,#Becurtovirus\n Viral replication is nuclear. ...,-1.916016,-3.111328,0.898926,2.949219,-4.785156
1,What is the role of the viral fiber glycoprote...,The viral fiber glycoproteins are involved in ...,The viral fiber glycoproteins code for 40 prot...,The viral fiber glycoproteins are responsible ...,The viral fiber glycoproteins mediate endocyto...,The viral fiber glycoproteins are responsible ...,D,D,Ichtadenovirus,https://en.wikipedia.org/wiki/Ichtadenovirus,...,NaN,1,first_order,0,#Ichtadenovirus\n Viruses in Ichtadenovirus ar...,-1.788086,-4.593750,-1.105469,2.556641,-3.951172
2,What is the significance of the faint Hα emiss...,The emission lines indicate that 3 Geminorum i...,The emission lines indicate that 3 Geminorum i...,The emission lines indicate that 3 Geminorum i...,The emission lines indicate that 3 Geminorum i...,The emission lines indicate that 3 Geminorum i...,A,A,3 Geminorum,https://en.wikipedia.org/wiki/3_Geminorum,...,NaN,2,first_order,0,"#3 Geminorum\n There is also a much fainter, a...",-4.687500,-2.925781,-2.373047,-3.150391,-4.500000
3,What is the significance of the pedicellariae ...,They are used for climbing on corals.,They resemble the traps of the Venus fly trap ...,They are covered by short and stout spines.,They are found on the central disc of the sea ...,They are a characteristic feature of the Gonia...,B,B,Hippasteria muscipula,https://en.wikipedia.org/wiki/Hippasteria_musc...,...,NaN,3,first_order,0,#Hippasteria muscipula\n Hippasteria muscipula...,-2.275391,2.544922,-2.447266,-1.720703,0.650879
4,What is the role of the microprocessor complex...,The microprocessor complex is responsible for ...,The microprocessor complex is responsible for ...,The microprocessor complex is involved in the ...,The microprocessor complex is involved in the ...,The microprocessor complex is responsible for ...,A,A,Microprocessor complex,https://en.wikipedia.org/wiki/Microprocessor_c...,...,NaN,4,first_order,0,#Microprocessor complex\nThe microprocessor co...,4.105469,-3.023438,-1.033203,-2.720703,-5.757812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,What is the relation between the three moment ...,The three moment theorem expresses the relatio...,The three moment theorem is used to calculate ...,The three moment theorem describes the relatio...,The three moment theorem is used to calculate ...,The three moment theorem is used to derive the...,C,C,NaN,NaN,...,195.0,495,first_order,5,#Theorem of three moments\n(Figure 02) Conside...,1.977539,-1.479492,4.597656,-3.218750,-0.510254
196,"What is the throttling process, and why is it ...",The throttling process is a steady flow of a f...,The throttling process is a steady adiabatic f...,The throttling process is a steady adiabatic f...,The throttling process is a steady flow of a f...,The throttling process is a steady adiabatic f...,B,B,NaN,NaN,...,196.0,496,first_order,5,#Refrigerator\n This liquid refrigerant is for...,0.770020,1.283203,2.447266,-0.327148,-1.945312
197,What happens to excess base metal as a solutio...,"The excess base metal will often solidify, bec...",The excess base metal will often crystallize-o...,"The excess base metal will often dissolve, bec...","The excess base metal will often liquefy, beco...","The excess base metal will often evaporate, be...",B,B,NaN,NaN,...,197.0,497,first_order,5,#Heat treating\n These two temperatures are ca...,7.046875,7.734375,4.171875,3.925781,3.130859
198,"What is t

In [20]:
def f(x):
    if x <= 300: 
        return "val" 
    else:
        return "test"
df_tests2["tta_id"] = df_tests2["data_id"].apply(f)

In [21]:
ret_scores = []
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
for group, w_df in df_tests2.groupby(["tta_id", "tta_no"]):
    ret_scores.append({
        "tta_id": group[0],
        "tta_no": group[1],
        "score": map_at_3(
            w_df[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]], 
            w_df["answer"].map(option_to_index)
        )
    })
pd.DataFrame(ret_scores)

,tta_id,tta_no,score
0,test,0,0.968174
1,test,1,0.964824
2,test,2,0.948074
3,test,3,0.958124
4,test,4,0.956449
5,test,5,0.945561
6,val,0,0.844961
7,val,1,0.808970
8,val,2,0.795127
9,val,3,0.803987


In [22]:
ret_scores = []
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
for group, w_df in df_tests2.groupby("tta_id"):
    ret_scores.append({
        "tta_id": group,
        "score": map_at_3(
            w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].max() + 
            w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].mean(), 
            w_df.groupby("data_id")["answer"].head(1).map(option_to_index).values
        )
    })
pd.DataFrame(ret_scores)

,tta_id,score
0,test,0.979899
1,val,0.828350


In [23]:
ret_scores = []
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
for group, w_df in df_tests2[df_tests2["tta_no"] <= 4].groupby("tta_id"):
    ret_scores.append({
        "tta_id": group,
        "score": map_at_3(
            w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].max() + 
            w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].mean(), 
            w_df.groupby("data_id")["answer"].head(1).map(option_to_index).values
        )
    })
pd.DataFrame(ret_scores)

,tta_id,score
0,test,0.982412
1,val,0.827243


In [24]:
ret_scores = []
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
for group, w_df in df_tests2[df_tests2["tta_no"] <= 3].groupby("tta_id"):
    ret_scores.append({
        "tta_id": group,
        "score": map_at_3(
            w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].max() + 
            w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].mean(), 
            w_df.groupby("data_id")["answer"].head(1).map(option_to_index).values
        )
    })
pd.DataFrame(ret_scores)

,tta_id,score
0,test,0.979899
1,val,0.836656


In [45]:
ret_scores = []
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
for group, w_df in df_tests2.groupby("tta_id"):
    ret_scores.append({
        "tta_id": group,
        "score": map_at_3(
            w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].max() * 2+ 
            w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].mean(), 
            w_df.groupby("data_id")["answer"].head(1).map(option_to_index).values
        )
    })
pd.DataFrame(ret_scores)

,tta_id,score
0,test,0.967337
1,val,0.837763


In [46]:
ret_scores = []
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
for group, w_df in df_tests2[df_tests2["tta_no"] <= 4].groupby("tta_id"):
    ret_scores.append({
        "tta_id": group,
        "score": map_at_3(
            w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].max() * 2 + 
            w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].mean(), 
            w_df.groupby("data_id")["answer"].head(1).map(option_to_index).values
        )
    })
pd.DataFrame(ret_scores)

,tta_id,score
0,test,0.967337
1,val,0.835548


In [47]:
ret_scores = []
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
for group, w_df in df_tests2[df_tests2["tta_no"] <= 3].groupby("tta_id"):
    ret_scores.append({
        "tta_id": group,
        "score": map_at_3(
            w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].max() * 2 + 
            w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].mean(), 
            w_df.groupby("data_id")["answer"].head(1).map(option_to_index).values
        )
    })
pd.DataFrame(ret_scores)

,tta_id,score
0,test,0.965662
1,val,0.829457


In [29]:
ret_scores = []
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
for group, w_df in df_tests2.groupby("tta_id"):
    ret_scores.append({
        "tta_id": group,
        "score": map_at_3(
            (
                w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].max() * 2/3 +
                w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].mean() * 1/3
            ),
            w_df.groupby("data_id")["answer"].head(1).map(option_to_index).values
        )
    })
pd.DataFrame(ret_scores)

,tta_id,score
0,first_order,0.889333


In [133]:
w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].nth(0)

,pred_A,pred_B,pred_C,pred_D,pred_E
data_id,,,,,
0,-8.921875,-8.734375,-2.863281,-8.453125,-8.718750
1,-7.160156,-4.304688,7.804688,-0.665039,-5.101562
2,-2.060547,-3.628906,-3.695312,6.296875,-4.785156
3,-7.398438,-7.671875,-4.269531,-8.875000,-0.409668
4,-3.525391,-3.458984,4.417969,-2.359375,2.728516
...,...,...,...,...,...
3895,-1.964844,-2.603516,4.457031,-5.929688,-4.792969
3896,-1.558594,-8.468750,-7.570312,-8.257812,-3.177734
3897,1.481445,0.339111,4.433594,-8.210938,-3.841797


In [182]:
ret_scores = []
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
for group, w_df in df_tests2[df_tests2["tta_no"] <= 5].groupby("tta_id"):
    ret_scores.append({
        "tta_id": group,
        "score": map_at_3(
            w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].max() + w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].mean(), 
            w_df.groupby("data_id")["answer"].head(1).map(option_to_index).values
        )
    })
pd.DataFrame(ret_scores)

,tta_id,score
0,first_order,0.884188
1,order,0.872607
2,skip_index,0.876154


In [193]:
ret_scores = []
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
for group, w_df in df_tests2[df_tests2["tta_no"] <= 5].groupby("tta_id"):
    ret_scores.append({
        "tta_id": group,
        "score": map_at_3(
            (
                w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].max() +
                w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].mean() 
            ),
            w_df.groupby("data_id")["answer"].head(1).map(option_to_index).values
        )
    })
pd.DataFrame(ret_scores)

,tta_id,score
0,first_order,0.884188
1,order,0.872607
2,skip_index,0.876154


In [192]:
ret_scores = []
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
for group, w_df in df_tests2[df_tests2["tta_no"] <= 5].groupby("tta_id"):
    ret_scores.append({
        "tta_id": group,
        "score": map_at_3(
            (
                w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].max() +
                w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].mean() +
                w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].min()
            ),
            w_df.groupby("data_id")["answer"].head(1).map(option_to_index).values
        )
    })
pd.DataFrame(ret_scores)

,tta_id,score
0,first_order,0.881624
1,order,0.865726
2,skip_index,0.869274


In [184]:
ret_scores = []
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
for group, w_df in df_tests2[df_tests2["tta_no"] <= 5].groupby("tta_id"):
    ret_scores.append({
        "tta_id": group,
        "score": map_at_3(
            (
                w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].max()*2/3 +
                w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].mean()*1/3
            ),
            w_df.groupby("data_id")["answer"].head(1).map(option_to_index).values
        )
    })
pd.DataFrame(ret_scores)

,tta_id,score
0,first_order,0.883675
1,order,0.873419
2,skip_index,0.878248


In [188]:
ret_scores = []
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
for group, w_df in df_tests2[df_tests2["tta_no"] <= 5].groupby("tta_id"):
    ret_scores.append({
        "tta_id": group,
        "score": map_at_3(
            (
                w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].max() * 2/3 +
                w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].mean() * 1/10 + 
                w_df[w_df["tta_no"] <= 1].groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].mean() * 1/3
            ),
            w_df.groupby("data_id")["answer"].head(1).map(option_to_index).values
        )
    })
pd.DataFrame(ret_scores)

,tta_id,score
0,first_order,0.883974
1,order,0.877137
2,skip_index,0.878248


In [194]:
ret_scores = []
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
for group, w_df in df_tests2[df_tests2["tta_no"] <= 3].groupby("tta_id"):
    ret_scores.append({
        "tta_id": group,
        "score": map_at_3(
            (
                w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].max() +
                w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].mean() 
            ),
            w_df.groupby("data_id")["answer"].head(1).map(option_to_index).values
        )
    })
pd.DataFrame(ret_scores)

,tta_id,score
0,first_order,0.882393
1,order,0.873547
2,skip_index,0.876068


In [197]:
ret_scores = []
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
for group, w_df in df_tests2[df_tests2["tta_no"] <= 3].groupby("tta_id"):
    ret_scores.append({
        "tta_id": group,
        "score": map_at_3(
            (
                w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].max() +
                w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].mean()
            ),
            w_df.groupby("data_id")["answer"].head(1).map(option_to_index).values
        )
    })
pd.DataFrame(ret_scores)

,tta_id,score
0,first_order,0.882393
1,order,0.873547
2,skip_index,0.876068


In [134]:
df_tests2

,Unnamed: 0,prompt,A,B,C,D,E,answer,source,dataset,...,searched_wiki_id_48,searched_wiki_id_49,data_id,tta_id,tta_no,pred_A,pred_B,pred_C,pred_D,pred_E
0,72133,What is the primary occupation of Javon Mascel...,Javon Mascellus Bullard is a renowned scientis...,Javon Mascellus Bullard is an acclaimed chef k...,Javon Mascellus Bullard is a professional Amer...,Javon Mascellus Bullard is a highly successful...,Javon Mascellus Bullard is an accomplished pia...,C,1,valid,...,#J. Marshall Shepherd\nJames Marshall Shepherd...,#Jabari Smith Jr.\n Smith began playing basket...,0,order,0,-8.921875,-8.734375,-2.863281,-8.453125,-8.718750
1,72134,How was Karnavati University established?,Karnavati University was established by the In...,Karnavati University was established through a...,Karnavati University was established in 2017 b...,Karnavati University was established through a...,Karnavati University was established through a...,C,1,valid,...,#Krantiguru Shyamji Krishna Verma Kachchh Univ...,#Krantiguru Shyamji Krishna Verma Kachchh Univ...,1,order,0,-7.160156,-4.304688,7.804688,-0.665039,-5.101562
2,72135,When was VA-144 established as VA-116?,23 February 1959,29 January 1959,1 December 1959,1 December 1955,29 January 1971,D,1,valid,...,#VFA-115\n The squadron received their first K...,#VA-122 (U.S. Navy)\n December 1969: The squad...,2,order,0,-2.060547,-3.628906,-3.695312,6.296875,-4.785156
3,72136,What was Lev Andreevich Navrozov primarily kno...,Lev Andreevich Navrozov was primarily known fo...,Lev Andreevich Navrozov was primarily known fo...,Lev Andreevich Navrozov was primarily known fo...,Lev Andreevich Navrozov was primarily known fo...,Lev Andreevich Navrozov was primarily known fo...,E,1,valid,...,#Russia\n Élie Metchnikoff is known for his gr...,#Lev Danilkin\n By the time he departed from A...,3,order,0,-7.398438,-7.671875,-4.269531,-8.875000,-0.409668
4,72137,What were the titles and positions held by Mic...,Michel Le Tellier held the titles of seigneur ...,Michel Le Tellier held the positions of seigne...,Michel Le Tellier held the title of marquis de...,Michel Le Tellier held the title of marquis de...,Michel Le Tellier held the title of marquis de...,C,1,valid,...,"#Michel Le Tellier\n He died in Versailles, 15...",#Jean de Forcade de Biaix\n and president of t...,4,order,0,-3.525391,-3.458984,4.417969,-2.359375,2.728516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3895,76028,Which logical fallacy is used in the text?\nMs...,circular reasoning: an argument that supports ...,guilt by association: a negative association i...,straw man: a misrepresentation of an opponent'...,a bicycle that moved 55miles north in 10hours,Do toy cars go faster down the ramp made of wo...,C,additional_data/ScienceQA/test.parquet,valid,...,#Invincible ignorance fallacy\nThe invincible ...,"#Argument\n In logic, an argument is usually e...",3895,skip_index,7,0.246338,-0.680176,5.425781,-6.253906,-3.292969
3896,76029,How long is a potato?,16 centimeters,16 kilometers,16 millimeters,16 meters,interrogative,A,additional_data/ScienceQA/test.parquet,valid,...,#Potato (disambiguation)\nLook up potato in Wi...,"#Marah (plant)\n They are large, and spherical...",3896,skip_index,7,-0.039001,-8.171875,-6.531250,-7.667969,-1.585938
3897,76030,Complete the statement. Assume that the motorc...,stayed the same,decreased,increased,Both are caused by heating.,a motorboat that moved 115kilometers east in 5...,C,additional_data/ScienceQA/test.parquet,valid,...,#Mass–energy equivalence\n If its temperature ...,#Einstein's thought experiments\n {\displaysty...,3897,skip_index,7,2.757812,-0.155029,5.238281,-7.210938,-2.224609
3898,76031,Which type of sentence is this?\nBecause most ...,simple,compound-complex,complex,compound,the ways the organism behaves,C,additional_data/ScienceQA/test.parquet,valid,...,"#Woods Cree\n For example, the sentence ""the c...",#English compound\n The term compound verb was...,3898,skip_inde

In [181]:
ret_scores = []
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
for group, w_df in df_tests2.groupby("tta_id"):
    ret_scores.append({
        "tta_id": group,
        "score": map_at_3(
            (
                w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].max() * 2/3 +
                w_df.groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].mean() * 1/10 + 
                w_df[w_df["tta_no"] <= 1].groupby("data_id")[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].mean() * 1/3
            ),
            w_df.groupby("data_id")["answer"].head(1).map(option_to_index).values
        )
    })
pd.DataFrame(ret_scores)

,tta_id,score
0,first_order,0.885513
1,order,0.876709
2,skip_index,0.878419


In [106]:

ret_scores = []
for th in [0, 1, 2, 3, 4, 5, 6]:
    predictions = []
    w_df = df_tests2[df_tests2["tta_id"] == "first_order"]
    for data_id, ww_df in tqdm.tqdm(w_df.groupby("data_id")):
        pred_final = ww_df.iloc[0][["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].values

        for i in range(len(ww_df)):
            pred = ww_df.iloc[i][["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].values
            if pred.max() > th:
                pred_final = pred
                continue
        predictions.append(pred_final)
    ret_scores.append({
        "tta_id": group,
        "th": th,
        "score": map_at_3(
            predictions, 
            w_df.groupby("data_id")["answer"].head(1).map(option_to_index).values
        )
    })
pd.DataFrame(ret_scores)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3900/3900 [00:15<00:00, 258.89it/s]


,tta_id,th,score
0,skip_index,0,0.864444
1,skip_index,1,0.868077
2,skip_index,2,0.871838
3,skip_index,3,0.875043
4,skip_index,4,0.875342
5,skip_index,5,0.876538
6,skip_index,6,0.875726


In [108]:

ret_scores = []
for th in [0, 1, 2, 3, 4, 5, 6]:
    predictions = []
    w_df = df_tests2[df_tests2["tta_id"] == "first_order"]
    for data_id, ww_df in tqdm.tqdm(w_df.groupby("data_id")):
        pred_final = ww_df[["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].max().values

        for i in range(len(ww_df)):
            pred = ww_df.iloc[i][["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].values
            if pred.max() > th:
                pred_final = pred
                continue
        predictions.append(pred_final)
    ret_scores.append({
        "tta_id": group,
        "th": th,
        "score": map_at_3(
            predictions, 
            w_df.groupby("data_id")["answer"].head(1).map(option_to_index).values
        )
    })
pd.DataFrame(ret_scores)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3900/3900 [00:15<00:00, 243.94it/s]


,tta_id,th,score
0,skip_index,0,0.863162
1,skip_index,1,0.867393
2,skip_index,2,0.871026
3,skip_index,3,0.875256
4,skip_index,4,0.877308
5,skip_index,5,0.879274
6,skip_index,6,0.880043


In [93]:
ww_df

,Unnamed: 0,prompt,A,B,C,D,E,answer,source,dataset,...,searched_wiki_id_48,searched_wiki_id_49,data_id,tta_id,tta_no,pred_A,pred_B,pred_C,pred_D,pred_E
4199,76332,Which of the following statements is the most ...,The Toba supereruption caused a volcanic winte...,The Toba supereruption had no impact on the cl...,Ash from the Toba supereruption in Lake Malawi...,The Toba supereruption caused a significant co...,The Toba supereruption had no effect on any cl...,C,20230924083304_gpt5,valid,...,#Toba catastrophe theory\n Computational ash d...,#Impact winter\n If the asteroid hit an ocean ...,4199,skip_index,0,-2.082031,-3.876953,3.318359,-2.525391,-2.984375
4199,76332,Which of the following statements is the most ...,The Toba supereruption caused a volcanic winte...,The Toba supereruption had no impact on the cl...,Ash from the Toba supereruption in Lake Malawi...,The Toba supereruption caused a significant co...,The Toba supereruption had no effect on any cl...,C,20230924083304_gpt5,valid,...,#Toba catastrophe theory\n Computational ash d...,#Impact winter\n If the asteroid hit an ocean ...,4199,skip_index,1,-0.141357,-1.822266,3.072266,-2.068359,-3.417969
4199,76332,Which of the following statements is the most ...,The Toba supereruption caused a volcanic winte...,The Toba supereruption had no impact on the cl...,Ash from the Toba supereruption in Lake Malawi...,The Toba supereruption caused a significant co...,The Toba supereruption had no effect on any cl...,C,20230924083304_gpt5,valid,...,#Toba catastrophe theory\n Computational ash d...,#Impact winter\n If the asteroid hit an ocean ...,4199,skip_index,2,-0.835449,-2.949219,3.462891,-1.401367,-2.455078
4199,76332,Which of the following statements is the most ...,The Toba supereruption caused a volcanic winte...,The Toba supereruption had no impact on the cl...,Ash from the Toba supereruption in Lake Malawi...,The Toba supereruption caused a significant co...,The Toba supereruption had no effect on any cl...,C,20230924083304_gpt5,valid,...,#Toba catastrophe theory\n Computational ash d...,#Impact winter\n If the asteroid hit an ocean ...,4199,skip_index,3,-0.525879,-5.128906,-0.990723,1.044922,-5.324219
4199,76332,Which of the following statements is the most ...,The Toba supereruption caused a volcanic winte...,The Toba supereruption had no impact on the cl...,Ash from the Toba supereruption in Lake Malawi...,The Toba supereruption caused a significant co...,The Toba supereruption had no effect on any cl...,C,20230924083304_gpt5,valid,...,#Toba catastrophe theory\n Computational ash d...,#Impact winter\n If the asteroid hit an ocean ...,4199,skip_index,4,-0.431885,-2.419922,3.884766,-1.599609,-2.185547
4199,76332,Which of the following statements is the most ...,The Toba supereruption caused a volcanic winte...,The Toba supereruption had no impact on the cl...,Ash from the Toba supereruption in Lake Malawi...,The Toba supereruption caused a significant co...,The Toba supereruption had no effect on any cl...,C,20230924083304_gpt5,valid,...,#Toba catastrophe theory\n Computational ash d...,#Impact winter\n If the asteroid hit an ocean ...,4199,skip_index,5,-1.055664,-1.759766,3.638672,-1.146484,-1.402344
4199,76332,Which of the following statements is the most ...,The Toba supereruption caused a volcanic winte...,The Toba supereruption had no impact on the cl...,Ash from the Toba supereruption in Lake Malawi...,The Toba supereruption caused a significant co...,The Toba supereruption had no effect on any cl...,C,20230924083304_gpt5,valid,...,#Toba catastrophe theory\n Computational ash d...,#Impact winter\n If the asteroid hit an ocean ...,4199,skip_index,6,-1.682617,-3.505859,3.482422,-2.968750,-3.251953
4199,76332,Which of the following statements is the most ...,The Toba supereruption caused a volcanic winte...,The Toba supereruption had no impact on the cl...,Ash from the Toba supereruption in Lake Malawi...,The Toba supereruption caused a significant co...,The Toba supereru

In [ ]:

ret_scores = []
predictions = []
th = 1
for group, w_df in df_tests2.groupby("tta_id"):
    for data_id, ww_df in tqdm.tqdm(w_df.groupby("data_id")):
        pred_final = ww_df.iloc[0][["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].values

        for i in range(len(ww_df)):
            pred = ww_df.iloc[i][["pred_A", "pred_B", "pred_C", "pred_D", "pred_E"]].values
            if pred.max() > th:
                pred_final = pred
                break
        predictions.append(pred_final)
    ret_scores.append({
        "tta_id": group,
        "score": map_at_3(
            predictions, 
            w_df.groupby("data_id")["answer"].head(1).map(option_to_index).values
        )
    })
pd.DataFrame(ret_scores)

,tta_id,tta_no,score
0,first_order,0,0.867738
1,first_order,1,0.855159
2,first_order,2,0.854643
3,first_order,3,0.855595
4,first_order,4,0.850000
5,first_order,5,0.850476
6,first_order,6,0.848254
7,first_order,7,0.845198
8,order,0,0.867738
9,order,1,0.796032


In [11]:
ret.append(calc_map3(
    model_name="../output/stage2/exp005.py/20230925235801_new_data_all300val_maxlen256_4epochs/checkpoint-13000/",
    max_length=384,
    num_content=3,
))

2023-09-26 06:42:17,023 INFO load data
2023-09-26 06:42:17,023 INFO load data
2023-09-26 06:42:17,023 INFO load data


  0%|          | 0/4200 [00:00<?, ?ex/s]

100%|███████████████████████████████████████| 4200/4200 [03:39<00:00, 19.14it/s]


0.8781349206349229


In [12]:
ret.append(calc_map3(
    model_name="../output/stage2/exp005.py/20230925235801_new_data_all300val_maxlen256_4epochs/checkpoint-13000/",
    max_length=512,
    num_content=4,
))

2023-09-26 06:46:27,481 INFO load data
2023-09-26 06:46:27,481 INFO load data
2023-09-26 06:46:27,481 INFO load data
2023-09-26 06:46:27,481 INFO load data


  0%|          | 0/4200 [00:00<?, ?ex/s]

100%|███████████████████████████████████████| 4200/4200 [04:36<00:00, 15.21it/s]


0.8803174603174629


In [13]:
ret.append(calc_map3(
    model_name="../output/stage2/exp005.py/20230925235801_new_data_all300val_maxlen256_4epochs/checkpoint-13000/",
    max_length=768,
    num_content=7,
))

2023-09-26 06:51:37,665 INFO load data
2023-09-26 06:51:37,665 INFO load data
2023-09-26 06:51:37,665 INFO load data
2023-09-26 06:51:37,665 INFO load data
2023-09-26 06:51:37,665 INFO load data


  0%|          | 0/4200 [00:00<?, ?ex/s]

100%|███████████████████████████████████████| 4200/4200 [09:32<00:00,  7.33it/s]


0.8750793650793676


In [15]:
df = pd.DataFrame(ret)

In [16]:
df

,model_name,max_length,num_content,map3,predictions
0,../output/stage2/exp005.py/20230925235801_new_...,256,2,0.871944,"[[-11.45, -9.97, -6.613, -8.914, -10.195], [-8..."
1,../output/stage2/exp005.py/20230925235801_new_...,384,3,0.878135,"[[-11.26, -10.39, -5.52, -8.484, -9.77], [-10...."
2,../output/stage2/exp005.py/20230925235801_new_...,512,4,0.880317,"[[-10.94, -10.12, -4.906, -7.586, -9.38], [-10..."
3,../output/stage2/exp005.py/20230925235801_new_...,768,7,0.875079,"[[-10.52, -9.93, -3.178, -6.89, -8.91], [-9.05..."


In [19]:
df_test = pd.read_parquet("../output/context_pipeline/stage1/exp009.py/20230925104920_gte-base_wikiall_without_sep_targetprompt_and_choice_without_sep_token_length120_stride_sentence4_drop_categoryTrue_all/valid.parquet")
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
label = df_test["answer"].map(option_to_index).values


In [20]:
ret2 = []
for i in range(len(df)):
    series = df.iloc[i]
    df_test_ = df_test.copy()
    df_test_["pred"] = [x for x in series["predictions"]]
    
    for source, w_df in df_test_.groupby("source"):
        print(source)
        option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
        label = w_df["answer"].map(option_to_index).values
        test_predictions = np.stack(w_df["pred"].values)
        ret2.append({
            "model_name": series["model_name"],
            "max_length": series["max_length"],
            "num_content": series["num_content"],
            "source": source,
            "map3": map_at_3(test_predictions, label)
        })

1
11
2
20230924083304_gpt5
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/ScienceQA/test.parquet
1
11
2
20230924083304_gpt5
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/ScienceQA/test.parquet
1
11
2
20230924083304_gpt5
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/ScienceQA/test.parquet
1
11
2
20230924083304_gpt5
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/ScienceQA/test.parquet


In [21]:
df_ret2 = pd.DataFrame(ret2)

In [22]:
df_ret2.sort_values(["model_name", "source", "max_length"])

,model_name,max_length,num_content,source,map3
0,../output/stage2/exp005.py/20230925235801_new_...,256,2,1,0.868889
14,../output/stage2/exp005.py/20230925235801_new_...,384,3,1,0.875556
28,../output/stage2/exp005.py/20230925235801_new_...,512,4,1,0.873889
42,../output/stage2/exp005.py/20230925235801_new_...,768,7,1,0.872222
1,../output/stage2/exp005.py/20230925235801_new_...,256,2,11,0.941667
15,../output/stage2/exp005.py/20230925235801_new_...,384,3,11,0.942778
29,../output/stage2/exp005.py/20230925235801_new_...,512,4,11,0.951111
43,../output/stage2/exp005.py/20230925235801_new_...,768,7,11,0.952778
2,../output/stage2/exp005.py/20230925235801_new_...,256,2,2,0.892778
16,../output/stage2/exp005.py/20230925235801_new_...,384,3,2,0.903889


In [23]:
df.to_pickle("20230926_ret.pickle")
df_ret2.to_pickle("20230926_ret2.pickle")

In [24]:
df_ret2.pivot_table(columns=["model_name", "source"], index="max_length", values="map3")

model_name ../output/stage2/exp005.py/20230925235801_new_data_all300val_maxlen256_4epochs/checkpoint-13000/  \
source                                                                                                    1   
max_length                                                                                                    
256                                                  0.868889                                                 
384                                                  0.875556                                                 
512                                                  0.873889                                                 
768                                                  0.872222                                                 

model_name                                                              \
source            11         2 20230924083304_gpt5         3         4   
max_length                                                               
256         0.941667  0.892778            0.789444  0.860000  0.880556   
384         0.942778  0.903889            0.798333  0.861111  0.887222   
512         0.951111  0.906111            0.787778  0.852778  0.901667   
768         0.952778  0.904444            0.789444  0.840556  0.882778   

model_name                                                    \
source             5         6         7         8         9   
max_length                                                     
256         0.936667  0.883333  0.880000  0.836667  0.795556   
384         0.932222  0.895556  0.884444  0.846667  0.815000   
512         0.938333  0.889444  0.897222  0.848333  0.828889   
768         0.918889  0.876111  0.885000  0.848889  0.827222   

model_name                                \
source     additional_data/MMLU/test.csv   
max_length                                 
256                             0.741667   
384                             0.745556   
512                             0.742778   
768                             0.755000   

model_name                                                                                  \
source     additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv   
max_length                                                                                   
256                                                  0.954444                                
384                                                  0.961111                                
512                                                  0.960000                                
768                                                  0.951111                                

model_name                                         
source     additional_data/ScienceQA/test.parquet  
max_length                                         
256                                      0.945556  
384                                      0.944444  
512                                      0.946111  
768                                      0.946667

In [25]:
df_ret2.pivot_table(index=["model_name", "max_length"], columns="source", values="map3")

source                                                                1  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.868889   
                                                   384         0.875556   
                                                   512         0.873889   
                                                   768         0.872222   

source                                                               11  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.941667   
                                                   384         0.942778   
                                                   512         0.951111   
                                                   768         0.952778   

source                                                                2  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.892778   
                                                   384         0.903889   
                                                   512         0.906111   
                                                   768         0.904444   

source                                                         20230924083304_gpt5  \
model_name                                         max_length                        
../output/stage2/exp005.py/20230925235801_new_d... 256                    0.789444   
                                                   384                    0.798333   
                                                   512                    0.787778   
                                                   768                    0.789444   

source                                                                3  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.860000   
                                                   384         0.861111   
                                                   512         0.852778   
                                                   768         0.840556   

source                                                                4  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.880556   
                                                   384         0.887222   
                                                   512         0.901667   
                                                   768         0.882778   

source                                                                5  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.936667   
                                                   384         0.932222   
                                                   512         0.938333   
                                                   768         0.918889   

source                                                                6  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.883333   
                                                   384         0.895556   
                                                   512         0.889444   
                                                   768         0.876111   

source                                                                7  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.880000   
                                                   384         0.884444   
                          

In [26]:
df_ret2.pivot_table(index=["model_name", "max_length"], columns="source", values="map3")

source                                                                1  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.868889   
                                                   384         0.875556   
                                                   512         0.873889   
                                                   768         0.872222   

source                                                               11  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.941667   
                                                   384         0.942778   
                                                   512         0.951111   
                                                   768         0.952778   

source                                                                2  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.892778   
                                                   384         0.903889   
                                                   512         0.906111   
                                                   768         0.904444   

source                                                         20230924083304_gpt5  \
model_name                                         max_length                        
../output/stage2/exp005.py/20230925235801_new_d... 256                    0.789444   
                                                   384                    0.798333   
                                                   512                    0.787778   
                                                   768                    0.789444   

source                                                                3  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.860000   
                                                   384         0.861111   
                                                   512         0.852778   
                                                   768         0.840556   

source                                                                4  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.880556   
                                                   384         0.887222   
                                                   512         0.901667   
                                                   768         0.882778   

source                                                                5  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.936667   
                                                   384         0.932222   
                                                   512         0.938333   
                                                   768         0.918889   

source                                                                6  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.883333   
                                                   384         0.895556   
                                                   512         0.889444   
                                                   768         0.876111   

source                                                                7  \
model_name                                         max_length             
../output/stage2/exp005.py/20230925235801_new_d... 256         0.880000   
                                                   384         0.884444   
                          

In [27]:
df_ret2[df_ret2["source"].isin(["1", "2", "3", "4", "5", "6", "7", "8"])].groupby(["model_name", "max_length"])["map3"].mean()

model_name                                                                                        max_length
../output/stage2/exp005.py/20230925235801_new_data_all300val_maxlen256_4epochs/checkpoint-13000/  256           0.879861
                                                                                                  384           0.885833
                                                                                                  512           0.888472
                                                                                                  768           0.878611
Name: map3, dtype: float64

In [55]:
df_ret2[df_ret2["source"].isin(["1", "2", "3", "4", "5", "6", "7", "8"])].groupby(["model_name", "max_length"])["map3"].mean()

model_name                                                                     max_length
../output/stage2/exp005.py/20230923195407_new_data_all300val_maxlen256/fold0/  256           0.859722
                                                                               384           0.864931
                                                                               512           0.862153
                                                                               768           0.863750
../output/stage2/exp005.py/20230924021914_new_data_all300val_maxlen384/fold0/  384           0.867014
                                                                               512           0.865417
                                                                               768           0.863264
../output/stage2/exp005.py/20230924084819_new_data_all300val_maxlen512/fold0/  512           0.867222
                                                                               768           0

In [56]:
df_ret2.groupby(["model_name", "max_length"])["map3"].mean()

model_name                                                                     max_length
../output/stage2/exp005.py/20230923195407_new_data_all300val_maxlen256/fold0/  256           0.867009
                                                                               384           0.869444
                                                                               512           0.867863
                                                                               768           0.869103
../output/stage2/exp005.py/20230924021914_new_data_all300val_maxlen384/fold0/  384           0.871111
                                                                               512           0.869573
                                                                               768           0.869060
../output/stage2/exp005.py/20230924084819_new_data_all300val_maxlen512/fold0/  512           0.872692
                                                                               768           0

In [72]:
df_test[df_test["A"].apply(len) < 30]["source"].value_counts()

11                                                                                 273
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv    261
9                                                                                  215
additional_data/ScienceQA/test.parquet                                             184
additional_data/MMLU/test.csv                                                      180
8                                                                                  143
7                                                                                  132
4                                                                                  100
3                                                                                   91
2                                                                                   69
5                                                                                   67
1                                          

In [69]:
ret3 = []
for i in range(len(df)):
    series = df.iloc[i]
    df_test_ = df_test.copy()
    df_test_["pred"] = [x for x in series["predictions"]]
    df_test_ = df_test_[df_test_["A"].apply(len) < 20]
    
    for source, w_df in df_test_.groupby("source"):
        print(source)
        option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
        label = w_df["answer"].map(option_to_index).values
        test_predictions = np.stack(w_df["pred"].values)
        ret3.append({
            "model_name": series["model_name"],
            "max_length": series["max_length"],
            "num_content": series["num_content"],
            "source": source,
            "map3": map_at_3(test_predictions, label)
        })

1
11
2
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/ScienceQA/test.parquet
1
11
2
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/ScienceQA/test.parquet
1
11
2
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/ScienceQA/test.parquet
1
11
2
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/ScienceQA/test.parquet
1
11
2
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/ScienceQA/test.parquet
1
11
2
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/Sci

In [70]:
df_ret3 = pd.DataFrame(ret3)

In [71]:
df_ret3.pivot_table(index=["model_name", "max_length"], columns="source", values="map3")

source                                                                1  \
model_name                                         max_length             
../output/stage2/exp005.py/20230923195407_new_d... 256         0.913194   
                                                   384         0.916667   
                                                   512         0.906250   
                                                   768         0.909722   
../output/stage2/exp005.py/20230924021914_new_d... 384         0.909722   
                                                   512         0.899306   
                                                   768         0.920139   
../output/stage2/exp005.py/20230924084819_new_d... 512         0.913194   
                                                   768         0.909722   

source                                                               11  \
model_name                                         max_length             
../output/stage2/exp005.py/20230923195407_new_d... 256         0.959302   
                                                   384         0.958010   
                                                   512         0.958656   
                                                   768         0.965116   
../output/stage2/exp005.py/20230924021914_new_d... 384         0.955426   
                                                   512         0.955426   
                                                   768         0.965116   
../output/stage2/exp005.py/20230924084819_new_d... 512         0.966408   
                                                   768         0.967054   

source                                                                2  \
model_name                                         max_length             
../output/stage2/exp005.py/20230923195407_new_d... 256         0.833333   
                                                   384         0.884615   
                                                   512         0.852564   
                                                   768         0.868590   
../output/stage2/exp005.py/20230924021914_new_d... 384         0.846154   
                                                   512         0.865385   
                                                   768         0.875000   
../output/stage2/exp005.py/20230924084819_new_d... 512         0.858974   
                                                   768         0.865385   

source                                                                3  \
model_name                                         max_length             
../output/stage2/exp005.py/20230923195407_new_d... 256         0.813853   
                                                   384         0.831169   
                                                   512         0.826840   
                                                   768         0.846320   
../output/stage2/exp005.py/20230924021914_new_d... 384         0.813853   
                                                   512         0.820346   
                                                   768         0.820346   
../output/stage2/exp005.py/20230924084819_new_d... 512         0.826840   
                                                   768         0.865801   

source                                                                4  \
model_name                                         max_length             
../output/stage2/exp005.py/20230923195407_new_d... 256         0.897287   
                                                   384         0.918605   
                                                   512         0.916667   
                                                   768         0.922481   
../output/stage2/exp005.py/20230924021914_new_d... 384         0.918605   
                                                   512         0.926357   
                                                   768         0.926357   
../output/stage2/exp0